# Kaggle Submission

In this notebook, I apply the steps I performed to clean the training dataset to the testing dataset so that I can submit my properly formatted predictions to Kaggle.

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from scipy import stats

### Data Cleaning & EDA

In [2]:
# Read the Kaggle test data
kaggle_test = pd.read_csv('../datasets/test.csv', na_values=[''], keep_default_na=False)

In [3]:
# Rename the columns
kaggle_test.columns = [column.lower().replace(' ','_') for column in kaggle_test.columns]
kaggle_test.head(1)

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NA,NA,NA,0,4,2006,WD


In [4]:
# Shape of dataset
kaggle_test.shape

(878, 80)

In [5]:
# Columns sorted by # of missing values (6 columns with missing values)
kaggle_test.isnull().sum().sort_values(ascending=False).head(6)

lot_frontage     160
garage_yr_blt     45
mas_vnr_area       1
garage_finish      1
electrical         1
mas_vnr_type       1
dtype: int64

In [6]:
# Kaggle test dtypes
kaggle_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               878 non-null    int64  
 1   pid              878 non-null    int64  
 2   ms_subclass      878 non-null    int64  
 3   ms_zoning        878 non-null    object 
 4   lot_frontage     718 non-null    float64
 5   lot_area         878 non-null    int64  
 6   street           878 non-null    object 
 7   alley            878 non-null    object 
 8   lot_shape        878 non-null    object 
 9   land_contour     878 non-null    object 
 10  utilities        878 non-null    object 
 11  lot_config       878 non-null    object 
 12  land_slope       878 non-null    object 
 13  neighborhood     878 non-null    object 
 14  condition_1      878 non-null    object 
 15  condition_2      878 non-null    object 
 16  bldg_type        878 non-null    object 
 17  house_style     

In [7]:
# 'lot_frontage' column check (continuous)
print(sorted(kaggle_test['lot_frontage'].unique()))
print(kaggle_test['lot_frontage'].dtypes)
print(kaggle_test['lot_frontage'].isnull().sum())

# Replace null with 0.0
kaggle_test['lot_frontage'].fillna(0.0,inplace=True)

# Check code execution
kaggle_test['lot_frontage'].isnull().sum()

[28.0, 69.0, nan, 21.0, 24.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 110.0, 112.0, 113.0, 114.0, 115.0, 116.0, 118.0, 120.0, 121.0, 122.0, 124.0, 126.0, 130.0, 131.0, 133.0, 136.0, 149.0, 150.0, 152.0, 160.0, 168.0, 174.0, 182.0]
float64
160


0

In [8]:
# 'garage_yr_blt' column check (discrete)
print(sorted(kaggle_test['garage_yr_blt'].unique()))
print(kaggle_test['garage_yr_blt'].dtypes)
print(kaggle_test['garage_yr_blt'].isnull().sum())

# Replace null with 'year_built' value
kaggle_test['garage_yr_blt'].fillna(kaggle_test['year_built'],inplace=True)

# Check code execution
kaggle_test['garage_yr_blt'].isnull().sum()

[1910.0, 1920.0, 1924.0, 1925.0, 1935.0, 1937.0, 1940.0, 1943.0, 1950.0, 1951.0, 1952.0, 1953.0, 1954.0, 1956.0, 1957.0, 1958.0, 1959.0, 1961.0, 1962.0, 1963.0, 1968.0, 1969.0, 1970.0, 1971.0, 1972.0, 1973.0, 1974.0, 1975.0, 1977.0, 1978.0, 1979.0, 1980.0, 1982.0, 1987.0, 1989.0, 1991.0, 1993.0, 1998.0, 1999.0, 2000.0, 2001.0, 2003.0, 2004.0, 2006.0, 2007.0, nan, 1900.0, 1906.0, 1908.0, 1915.0, 1916.0, 1917.0, 1918.0, 1921.0, 1922.0, 1923.0, 1926.0, 1927.0, 1928.0, 1930.0, 1931.0, 1932.0, 1934.0, 1938.0, 1939.0, 1941.0, 1946.0, 1947.0, 1948.0, 1949.0, 1955.0, 1960.0, 1964.0, 1965.0, 1966.0, 1967.0, 1976.0, 1981.0, 1983.0, 1984.0, 1985.0, 1986.0, 1988.0, 1990.0, 1992.0, 1994.0, 1995.0, 1996.0, 1997.0, 2002.0, 2005.0, 2008.0, 2009.0, 2010.0]
float64
45


0

In [9]:
# 'mas_vnr_type' column check (nominal)
print(kaggle_test['mas_vnr_type'].unique())
print(kaggle_test['mas_vnr_type'].dtypes)
print(kaggle_test['mas_vnr_type'].isnull().sum())

# Replace null with 'None'
kaggle_test['mas_vnr_type'].fillna('None',inplace=True)

# Check code execution
kaggle_test['mas_vnr_type'].isnull().sum()

['None' 'BrkFace' 'Stone' 'BrkCmn' 'CBlock' nan]
object
1


0

In [10]:
# 'mas_vnr_area' column check (continuous)
print(sorted(kaggle_test['mas_vnr_area'].unique()))
print(kaggle_test['mas_vnr_area'].dtypes)
print(kaggle_test['mas_vnr_area'].isnull().sum())

# Replace null with 0.0
kaggle_test['mas_vnr_area'].fillna(0.0,inplace=True)

# Check code execution
kaggle_test['mas_vnr_area'].isnull().sum()

[0.0, 11.0, 14.0, 16.0, 18.0, 20.0, 23.0, 32.0, 34.0, 44.0, 45.0, 48.0, 50.0, 51.0, 52.0, 53.0, 54.0, 58.0, 60.0, 65.0, 66.0, 67.0, 68.0, 70.0, 72.0, 74.0, 75.0, 76.0, 80.0, 81.0, 82.0, 84.0, 85.0, 88.0, 89.0, 90.0, 91.0, 94.0, 95.0, 96.0, 98.0, 99.0, 100.0, 101.0, 104.0, 106.0, 108.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 120.0, 123.0, 128.0, 130.0, 134.0, 136.0, 137.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 147.0, 149.0, 150.0, 151.0, 153.0, 156.0, 161.0, 162.0, 164.0, 166.0, 169.0, 170.0, 171.0, 172.0, 174.0, 175.0, 176.0, 178.0, 180.0, 182.0, 187.0, 188.0, 189.0, 190.0, 194.0, 196.0, 198.0, 200.0, 203.0, 205.0, 206.0, 207.0, 208.0, 209.0, 212.0, 215.0, 216.0, 217.0, 218.0, 220.0, 223.0, 226.0, 227.0, 229.0, 230.0, 232.0, 233.0, 236.0, 238.0, 240.0, 246.0, 247.0, 248.0, 250.0, 252.0, 256.0, 258.0, 259.0, 260.0, 261.0, 263.0, 264.0, 265.0, 266.0, 268.0, 270.0, 280.0, 281.0, 284.0, 285.0, 286.0, 287.0, 288.0, 293.0, 295.0, 297.0, 300.0, 302.0, 304.0, 305.0, 306.0, 308.0, 310.0,

0

In [11]:
# 'garage_finish' column check (ordinal)
# Replace null with 'NA'
kaggle_test['garage_finish'].fillna('NA',inplace=True)
kaggle_test['garage_finish'].unique()

array(['Unf', 'Fin', 'RFn', 'NA'], dtype=object)

In [12]:
# 'electrical' column check (ordinal)
kaggle_test['electrical'].fillna('Mix',inplace=True)
kaggle_test['electrical'].unique()

array(['FuseP', 'SBrkr', 'FuseA', 'FuseF', 'Mix'], dtype=object)

In [13]:
# Check missing values
kaggle_test.isnull().sum().sort_values(ascending=False).head()

sale_type         0
yr_sold           0
year_remod/add    0
roof_style        0
roof_matl         0
dtype: int64

In [14]:
# Outliers - check 'garage_yr_blt'
kaggle_test.describe().T

,count,mean,std,min,25%,50%,75%,max
id,878.0,1.444749e+03,8.508383e+02,2.0,6.922500e+02,1433.0,2.197500e+03,2.928000e+03
pid,878.0,7.162867e+08,1.889103e+08,526302110.0,5.284841e+08,535454155.0,9.071904e+08,1.007100e+09
ms_subclass,878.0,5.826879e+01,4.223541e+01,20.0,2.000000e+01,50.0,7.000000e+01,1.900000e+02
lot_frontage,878.0,5.687244e+01,3.426950e+01,0.0,4.000000e+01,60.0,7.800000e+01,1.820000e+02
lot_area,878.0,1.030703e+04,1.000267e+04,1477.0,7.297250e+03,9446.0,1.158900e+04,2.152450e+05
overall_qual,878.0,6.050114e+00,1.369065e+00,2.0,5.000000e+00,6.0,7.000000e+00,1.000000e+01
overall_cond,878.0,5.566059e+00,1.128903e+00,1.0,5.000000e+00,5.0,6.000000e+00,9.000000e+00
year_built,878.0,1.970492e+03,3.039589e+01,1880.0,1.954000e+03,1972.0,1.999750e+03,2.010000e+03
year_remod/add,878.0,1.984418e+03,2.045072e+01,1950.0,1.967000e+03,1992.0,2.003000e+03,2.010000e+03
mas_vnr_area,878.0,1.061150e+02,1.870865e+02,0.0,0.000000e+00,0.0,1.717500e+02,1.378000e+03


In [15]:
# Dtype conversions

# 'pid' (int to nominal)
kaggle_test['pid'] = kaggle_test['pid'].astype(str)

# 'ms_subclass' (int to nominal)
kaggle_test['ms_subclass'] = kaggle_test['ms_subclass'].astype(str)

# 'lot_shape' (object to ordinal)
lot_shape_dict = {'IR3': 1, 'IR2': 2, 'IR1': 3, 'Reg': 4}
kaggle_test['lot_shape'] = kaggle_test['lot_shape'].map(lot_shape_dict)

# 'utilities' (object to ordinal)
utilities_dict = {'ELO': 1,'NoSeWa': 2, 'NoSewr': 3, 'AllPub': 4}
kaggle_test['utilities'] = kaggle_test['utilities'].map(utilities_dict)

# 'land_slope' (object to ordinal)
land_slope_dict = {'Sev': 1, 'Mod': 2, 'Gtl': 3}
kaggle_test['land_slope'] = kaggle_test['land_slope'].map(land_slope_dict)

# 'exter_qual' (object to ordinal)
exter_qual_dict = {'Po': 1,'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
kaggle_test['exter_qual'] = kaggle_test['exter_qual'].map(exter_qual_dict)

# 'exter_cond' (object to ordinal)
exter_cond_dict = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
kaggle_test['exter_cond'] = kaggle_test['exter_cond'].map(exter_cond_dict)

# 'bsmt_qual' (object to ordinal)
bsmt_qual_dict = {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
kaggle_test['bsmt_qual'] = kaggle_test['bsmt_qual'].map(bsmt_qual_dict)

# 'bsmt_cond' (object to ordinal)
bsmt_cond_dict = {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
kaggle_test['bsmt_cond'] = kaggle_test['bsmt_cond'].map(bsmt_cond_dict)

# 'bsmt_exposure' (object to ordinal)
bsmt_exposure_dict = {'NA': 0, 'No': 0, 'Mn': 1, 'Av': 2, 'Gd': 3}
kaggle_test['bsmt_exposure'] = kaggle_test['bsmt_exposure'].map(bsmt_exposure_dict)

# 'bsmtfin_type_1' (object to ordinal)
bsmtfin_type_1_dict = {'NA':0,'Unf':1,'LwQ':2,'Rec':3,'BLQ':4,'ALQ':5,'GLQ':6}
kaggle_test['bsmtfin_type_1'] = kaggle_test['bsmtfin_type_1'].map(bsmtfin_type_1_dict)

# 'bsmtfin_type_2' (object to ordinal)
bsmtfin_type_2_dict = {'NA':0,'Unf':1,'LwQ':2,'Rec':3,'BLQ':4,'ALQ':5,'GLQ':6}
kaggle_test['bsmtfin_type_2'] = kaggle_test['bsmtfin_type_2'].map(bsmtfin_type_2_dict)

# 'heating_qc' (object to ordinal)
heating_qc_dict = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
kaggle_test['heating_qc'] = kaggle_test['heating_qc'].map(heating_qc_dict)

# 'electrical' (object to ordinal)
electrical_dict = {'Mix': 1, 'FuseP': 2, 'FuseF': 3, 'FuseA':4, 'SBrkr':5}
kaggle_test['electrical'] = kaggle_test['electrical'].map(electrical_dict)

# 'kitchen_qual' (object to ordinal)
kitchen_qual_dict = {'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
kaggle_test['kitchen_qual'] = kaggle_test['kitchen_qual'].map(kitchen_qual_dict)

# 'functional' (object to ordinal)
functional_dict = {'Sal':1,'Sev':2,'Maj2':3,'Maj1':4,'Mod':5,'Min2':6,'Min1':7,'Typ':8}
kaggle_test['functional'] = kaggle_test['functional'].map(functional_dict)

# 'fireplace_qu' (object to ordinal)
fireplace_qu_dict = {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
kaggle_test['fireplace_qu'] = kaggle_test['fireplace_qu'].map(fireplace_qu_dict)

# 'garage_finish' (object to ordinal)
garage_finish_dict = {'NA':0,'Unf':1,'RFn':2,'Fin':3}
kaggle_test['garage_finish'] = kaggle_test['garage_finish'].map(garage_finish_dict)

# 'garage_qual' (object to ordinal)
garage_qual_dict = {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
kaggle_test['garage_qual'] = kaggle_test['garage_qual'].map(garage_qual_dict)

# 'garage_cond' (object to ordinal)
garage_cond_dict = {'NA': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
kaggle_test['garage_cond'] = kaggle_test['garage_cond'].map(garage_cond_dict)

# 'paved_drive' (object to ordinal)
paved_drive_dict = {'N':0,'P':1,'Y':2}
kaggle_test['paved_drive'] = kaggle_test['paved_drive'].map(paved_drive_dict)

# 'pool_qc' (object to ordinal)
pool_qc_dict = {'NA': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}
kaggle_test['pool_qc'] = kaggle_test['pool_qc'].map(pool_qc_dict)

# 'fence' (object to ordinal)
fence_dict = {'NA':0,'MnWw':1,'GdWo':2,'MnPrv':3,'GdPrv':4}
kaggle_test['fence'] = kaggle_test['fence'].map(fence_dict)

# 'neighborhood' (object to ordinal)
# Map 'neighborhood' based on 'saleprice' quartiles and create a new column 'mod_neighborhood'
neighborhood_dict = {'MeadowV': 1, 'IDOTRR': 1, 'BrDale': 1, 'OldTown': 1, 'BrkSide': 1, 'Edwards': 1, 
                     'SWISU': 2, 'Landmrk': 2, 'Sawyer': 2, 'NPkVill': 2, 'Blueste': 2, 'NAmes': 2, 
                     'Mitchel': 3, 'SawyerW': 3, 'Greens': 3, 'Gilbert': 3, 'NWAmes': 3, 'Blmngtn': 3, 'CollgCr': 3, 'Crawfor': 3, 
                     'StoneBr': 4, 'NridgHt': 4, 'NoRidge': 4, 'GrnHill': 4, 'Veenker': 4, 'Timber': 4, 'Somerst': 4, 'ClearCr': 4}
kaggle_test['mod_neighborhood'] = kaggle_test['neighborhood'].map(neighborhood_dict)

In [16]:
# Final check of missing values
kaggle_test.isnull().sum().sort_values(ascending=False).head()

mod_neighborhood    0
total_bsmt_sf       0
year_remod/add      0
roof_style          0
roof_matl           0
dtype: int64

### Model Submission

In [17]:
# Read 'cleaned_data.csv'
cleaned_data = pd.read_csv('../datasets/cleaned_data.csv')

In [18]:
# Model 4: Ridge Regression with Additional Features

# Define X and y
features = ['mod_neighborhood','overall_qual', 'exter_qual', 'gr_liv_area', 'kitchen_qual', 
            'garage_area', 'total_bsmt_sf', '1st_flr_sf', 'bsmt_qual', 'year_built', 
            'full_bath', 'garage_finish', 'totrms_abvgrd', 'bsmt_exposure']
X = cleaned_data[features]
y = cleaned_data['saleprice']

# Instantiate and fit PolynomialFeatures (i.e. feature engineering)
pf = PolynomialFeatures(degree=2,include_bias=False)
X_pf = pf.fit_transform(X)

# train/test split
X_train,X_test,y_train,y_test = train_test_split(X_pf,y,random_state=42)

# Scale the data with StandardScaler
ss = StandardScaler()
Z_train = ss.fit_transform(X_train)
Z_test = ss.transform(X_test)

# Ridge - find the optimal alpha
r_alphas = np.logspace(0,5,100)
ridge_cv = RidgeCV(alphas=r_alphas,scoring='r2',cv=5)
ridge_cv.fit(Z_train,y_train)

# Ridge - optimal alpha
ridge_cv.alpha_

# Instantiate and fit Ridge using the optimal alpha
ridge = Ridge(alpha=ridge_cv.alpha_)
ridge.fit(Z_train,y_train)
ridge.score(Z_train,y_train), ridge.score(Z_test,y_test)

(0.900819156089216, 0.9087723186165432)

In [19]:
# Predicted y values
y_pred = ridge.predict(Z_test)

# RMSE for training, testing data
np.sqrt(metrics.mean_squared_error(y_train,ridge.predict(Z_train))), np.sqrt(metrics.mean_squared_error(y_test,y_pred))

(24795.514697789768, 24410.759722702358)

In [20]:
# Poly transform Kaggle test data 
kaggle_pf = pf.transform(kaggle_test[features])

# Standardize the data
kaggle_ss = ss.transform(kaggle_pf)

# Kaggle predictions
kaggle_preds = ridge.predict(kaggle_ss)

# Create a new column in kaggle_test for 'saleprice' predictions
kaggle_test['saleprice'] = kaggle_preds

# Submission dictionary
preds_dict = {
    'id' : kaggle_test['id'],
    'saleprice' : kaggle_test['saleprice']
}

# Submission dataframe
# preds_df = pd.DataFrame(preds_dict)
preds_df = kaggle_test[['id', 'saleprice']]

# Save as submission file
preds_df.to_csv('../datasets/sw_final_submission.csv', index=False)